In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
# from model import build_unet
# from metrics import dice_loss, dice_coef, iou

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(2, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

In [ ]:
H = 512
W = 768

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = x[:,:,0:2] # Discard the last channel 

    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 2])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/sunil/icvgip/fundal_image/
!ls

/content/gdrive/MyDrive/sunil/icvgip/fundal_image
drive.zip      files_od_and_bv_weighted_demo_1	  HRF.zip    trained_models
files	       fundal_image-1.0-py3-none-any.whl  logs
files_demo_bv  HRF				  new_data
files_demo_od  hrf_resized			  new_data4


In [ ]:
#!unzip new_data2.zip

In [ ]:
model_dir = "trained_models"
model_name = "model_hrf_combined"
result_dir = "hrf_combined_results"

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)
create_dir(result_dir)

""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 30
model_path = os.path.join(model_dir, f"{model_name}.h5")
csv_path = os.path.join(result_dir, "training_history.csv")

""" Dataset """
dataset_path = "hrf_resized"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
# model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
# model.summary()

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 60 - 60
Valid: 5 - 5
Epoch 1/30
30/30 [==============================] - 72s 328ms/step - loss: 0.4816 - dice_coef: 0.1595 - iou: 0.0868 - recall_2: 0.2316 - precision_2: 0.5461 - val_loss: 0.6610 - val_dice_coef: 0.1047 - val_iou: 0.0553 - val_recall_2: 0.0012 - val_precision_2: 0.2330

Epoch 00001: val_loss improved from inf to 0.66102, saving model to trained_models/model_hrf_combined.h5
Epoch 2/30
30/30 [==============================] - 7s 240ms/step - loss: 0.3151 - dice_coef: 0.1892 - iou: 0.1046 - recall_2: 0.0817 - precision_2: 0.9335 - val_loss: 0.7165 - val_dice_coef: 0.0983 - val_iou: 0.0517 - val_recall_2: 0.3932 - val_precision_2: 0.2826

Epoch 00002: val_loss did not improve from 0.66102
Epoch 3/30
30/30 [==============================] - 7s 241ms/step - loss: 0.2712 - dice_coef: 0.2272 - iou: 0.1283 - recall_2: 0.1015 - precision_2: 0.9725 - val_loss: 0.4878 - val_dice_coef: 0.0986 - val_iou: 0.0519 - val_recall_2: 1.4497e-05 - val_precision_2: 0.0265

Epoch 0000

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

H = 512
W = 768

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = x[:,:,0:2] # Discard the last channel 
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path, channel):
    line = np.ones((y_pred.shape[0], 10, 3)) * 255

    pred_image = np.zeros((y_pred.shape[0], y_pred.shape[1], 3))
    _y_pred = y_pred[:, :, channel]
    _ori_y = ori_y[:, :, channel]
    pred_image[:, :, 0] = ((_y_pred > 0.5) & (_ori_y <= 128)) * 255
    pred_image[:, :, 1] = ((_y_pred > 0.5) & (_ori_y  > 128)) * 255
    pred_image[:, :, 2] = ((_ori_y  > 128) & (_y_pred <= 0.5 )) * 255

    print(" saving result", save_image_path)
    cv2.imwrite(save_image_path, pred_image)


In [ ]:
def get_filenames_sorted(image_path, gt_path=None):
    image_file_names = image_files(image_path)
    gt_file_names = None
    if gt_path is not None:
        # Get the test files also
        gt_file_names = sorted(image_files(gt_path))
    return image_file_names, gt_file_names

In [ ]:
def image_files(path, file_prefix=None):
    all_image_files = []
    if file_prefix is None:
        file_names = list(glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        print(prefix___)
        file_names = list(glob(prefix___))


    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext in supported_extensions:
            all_image_files.append(filename)
    return sorted(all_image_files)


In [ ]:
data_dir = "hrf_resized"
od_channel, bv_channel = 0, 1

od_result_dir = result_dir + "/od_predictions"
bv_result_dir = result_dir + "/bv_predictions"

create_dir(od_result_dir)
create_dir(bv_result_dir)

""" Load the model """
model_file_name = f"{model_dir}/{model_name}.h5"
print(model_file_name)
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model(model_file_name)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif"]

test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

""" Make the prediction and calculate the metrics values """
SCORE_BV, SCORE_OD = [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_bv = f"{bv_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_bv, bv_channel)


#     """ Calculate the bv metrics """
    bv_pred = y_pred[:, :, bv_channel].flatten()
    bv_gt = y[:, :, bv_channel].flatten()
    acc_value = accuracy_score(bv_gt > 0.5, bv_pred>0.5)
    f1_value = f1_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > 0.5) & (bv_pred > 0.5)) / np.sum(bv_gt > 0.5)
    precision_value = precision_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    SCORE_BV.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

#     """ Calculate the od metrics """
    bv_pred = y_pred[:, :, od_channel].flatten()
    bv_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(bv_gt > 0.5, bv_pred>0.5)
    f1_value = f1_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > 0.5) & (bv_pred > 0.5)) / np.sum(bv_gt > 0.5)
    precision_value = precision_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])
    
print("\n")
for SCORE in [SCORE_OD, SCORE_BV]:
    if SCORE == SCORE_OD:
        print("****** OD Metrics")
    else:
        print("****** BV Metrics")
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f} (dice score)")
    print(f"AUC: {score[1]:0.5f} (Auc score)")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision"])
        df.to_csv(f"{od_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision"])
        df.to_csv(f"{bv_result_dir}/score.csv")
    print("\n")

trained_models/model_hrf_combined.h5


  0%|          | 0/5 [00:00<?, ?it/s]

11_h
255 1.0
 saving result hrf_combined_results/od_predictions/11_h.png
 saving result hrf_combined_results/bv_predictions/11_h.png


 20%|██        | 1/5 [00:01<00:05,  1.31s/it]

12_h
255 1.0
 saving result hrf_combined_results/od_predictions/12_h.png
 saving result hrf_combined_results/bv_predictions/12_h.png


...

14_h
255 1.0
 saving result hrf_combined_results/od_predictions/14_h.png
 saving result hrf_combined_results/bv_predictions/14_h.png


 80%|████████  | 4/5 [00:04<00:01,  1.12s/it]

15_h
255 1.0
 saving result hrf_combined_results/od_predictions/15_h.png
 saving result hrf_combined_results/bv_predictions/15_h.png


100%|██████████| 5/5 [00:05<00:00,  1.08s/it]



****** OD Metrics
Accuracy: 0.99494
F1: 0.72737 (dice score)
AUC: 0.72737 (Auc score)
Jaccard: 0.57804
Recall: 0.58043
Precision: 0.99347


****** BV Metrics
Accuracy: 0.96322
F1: 0.78299 (dice score)
AUC: 0.78299 (Auc score)
Jaccard: 0.64572
Recall: 0.76218
Precision: 0.81338


